# DESGLOSE INITIAL APPROACH MINIMAL MEDIA

El objetivo es reorganizar la tabla obtenida del análisis anterior, de manera que se incluya una columna para cada metabolito. De esta manera, para cada organismo se indicará si el medio contiene un metabolito dado en función del valor de su columna:
- 1: lo contiene
- 0: no lo contiene

In [1]:
import openpyxl
import requests
import cobra
import os
from bs4 import BeautifulSoup
from cobra.io import read_sbml_model
from pathlib import Path
from cobra.medium import minimal_medium
import warnings
import json

## Importamos resultado

In [2]:
#Importamos los resultados
with open('Mejores_especies_minimal.json', 'r') as archivo:
    resultados = json.load(archivo)

In [3]:
resultados

{'Bacillus_amyloliquefaciens': {'Nombre': 'Bacillus_amyloliquefaciens_subsp_amyloliquefaciens_DC_12.xml',
  'Medio': ['1,2-Diacyl-sn-glycerol (dioctadecanoyl, n-C18:0)',
   '4-hydroxybenzoate',
   'Adenosine',
   'L-alanyl-L-histidine',
   'L-arginine',
   'calcium(2+)',
   'L-cysteinylglycine',
   'Chloride',
   'Co2+',
   'Cu2+',
   'Deoxyguanosine',
   'Fe3+',
   'D-glucose',
   'Glycerol 3-phosphate',
   'glycogen, structure 2 (glycogenin-1,6-{7[1,4-Glc], 4[1,4-Glc]})',
   'potassium',
   'L-methionyl-L-alanine',
   'magnesium',
   'Mn2+',
   'O2',
   'Protoheme',
   'Riboflavin',
   'sulfate',
   'Thiamin',
   'aldehydo-D-xylose',
   'Zinc'],
  'Tamaño_medio': 26,
  'Crecimiento': 30.746387429716346},
 'Bacillus_subtilis': {'Nombre': 'Bacillus_subtilis_subsp_subtilis_str_SMY.xml',
  'Medio': ['1,2-Diacyl-sn-glycerol (dioctadecanoyl, n-C18:0)',
   '4-hydroxybenzoate',
   'L-alanyl-L-threonine',
   'L-arginine',
   'calcium(2+)',
   'L-cysteinylglycine',
   'Chloride',
   'Co2+',
  

## Nombres de las columnas

Los nombres de las columnas serán todos lo metabolitos diferentes que encontremos en los diferentes medios de todos los organismos.

In [4]:
metab_unique = [] #Lista de todos los metabolitos diferentes de todos los medios

for especie, info_especie in resultados.items():
    modelo = info_especie["Nombre"]
    medio = info_especie["Medio"]
    tamaño = info_especie["Tamaño_medio"]
    crecimiento = info_especie["Crecimiento"]
    for met in medio:
        if met not in metab_unique:
            metab_unique.append(met)

In [5]:
metab_unique

['1,2-Diacyl-sn-glycerol (dioctadecanoyl, n-C18:0)',
 '4-hydroxybenzoate',
 'Adenosine',
 'L-alanyl-L-histidine',
 'L-arginine',
 'calcium(2+)',
 'L-cysteinylglycine',
 'Chloride',
 'Co2+',
 'Cu2+',
 'Deoxyguanosine',
 'Fe3+',
 'D-glucose',
 'Glycerol 3-phosphate',
 'glycogen, structure 2 (glycogenin-1,6-{7[1,4-Glc], 4[1,4-Glc]})',
 'potassium',
 'L-methionyl-L-alanine',
 'magnesium',
 'Mn2+',
 'O2',
 'Protoheme',
 'Riboflavin',
 'sulfate',
 'Thiamin',
 'aldehydo-D-xylose',
 'Zinc',
 'L-alanyl-L-threonine',
 'cytosine',
 'Fe2+',
 'Glycyl-L-asparagine',
 'L-lysine',
 'Nicotinate',
 'pectins',
 'meso-2,6-Diaminoheptanedioate',
 'L-alanyl-L-leucine',
 'D-Glucosamine',
 'Glycylleucine',
 'Glycyl-L-tyrosine',
 'hydrogenphosphate',
 'Spermidine',
 'Adenosylcobalamin',
 'Larch arabinogalactan',
 'Glycyl-L-glutamine',
 'Glycyl-L-methionine',
 'Xanthine',
 'Cytidine',
 'Gly-Cys',
 'L-alanyl-L-glutamine',
 'Menaquinone 7',
 '(R)-Pantothenate',
 'Pyridoxal',
 'Guanine',
 'Hypoxanthine',
 'Nitrate

## Creación del Excel

Creamos el excel final:

In [6]:
# Crear un nuevo libro de Excel y seleccionar la hoja de trabajo
workbook = openpyxl.load_workbook("Desglose_Initial_Approach_Minimal.xlsx")
hoja = workbook.active

# Rellenamos por cada especie(fila):
# Columna 1 con el nombre del mejor modelo
# Columna 2 tamaño del medio
# Columna 3 crecimiento
# Columnas 4-n metabolitos del medio

#Añadimos los nombres de las n columnas de metabolitos
for i, met  in enumerate(metab_unique, start=4):
    hoja.cell(row=1, column=i, value=met)

#Rellenamos las columnas
for i, nombre  in enumerate(resultados.keys(), start=2):
    info_especie = resultados[nombre] #Diccionario de la especie
    hoja.cell(row=i, column=1, value=info_especie["Nombre"])
    hoja.cell(row=i, column=2, value=info_especie["Tamaño_medio"])
    hoja.cell(row=i, column=3, value=info_especie["Crecimiento"])
    #Recorremos las columnas de los metabolitos
    for j, met in enumerate(metab_unique, start=4):
        if met in info_especie["Medio"]:
            hoja.cell(row=i, column=j, value=1)
        else:
            hoja.cell(row=i, column=j, value=0)

# Guardar el libro de Excel
workbook.save('Desglose_Initial_Approach_Minimal.xlsx')